## Explore the time-series of metrics in the control group

#### Aim

Explore the characteristics of the time-series of the main evaluation metrics (in terms of trends and seasonality) in the control group: those visitors who, on a given date, only visited pages containing no node2vec-generated related links.

The assumption is that their behaviours should not be affected by our intervention but, if we see trends, these could be the result of confounders that are shared with the intervention time-series, like the Explore Team's changes to the main menu and navigation or increase user needs for - e.g. - coronavirus information.

#### Background

The time-series data was created by the `notebooks/2021_tests/<ADD>.sql` for the time period currently available covering both A and B segments from 11-Oct-2021 to 15-Dec-2021 and saved in BigQuery data in the `govuk-bigquery-analytics.datascience.control_manual_links_20211011_20211215_data` table.

**IMPORTANT**: This notebook uses an R kernel.

#### How to setup Jupyter Notebook for R

These instructions assume that you already have a working Python environment for your local repository of this project, and Jupyter Notebook already installed in that environment that you can execute from your Terminal.

1. Install R 

   If not already installed, see https://cloud.r-project.org/index.html
   

2. Install R kernel for Jupyter Notebook

    In your Terminal (note: not in RStudio, not in the R GUI):
    
    - launch R by entering `R` on the command line.

    - You should now be using R from your Terminal. Thus, run:
    ```
    install.packages('IRkernel')
    IRkernel::installspec()
    ```

    Done! You can now quit R by entering `q()`.

If you now launch Jupyter Notebook, you'll have the option to choose `R` as kernel.


### Setting things up

In [ ]:
# Install packages, if they aren't already available.
# This can take a minute or two.
packages <- c("bigrquery", "tidyverse", "plotly", "gridExtra", 
              "tsibble", "feasts", "DT", "TTR", "lmtest", "Epi", "tsModel", "sandwich")
install.packages(setdiff(packages, rownames(installed.packages())), quiet = TRUE) 

In [ ]:
for(pckg in packages){
    suppressPackageStartupMessages(library(pckg, character.only = TRUE))
}

In [ ]:
#Authenticate
#/path/to/your/service-account.json
bq_auth(path = "/Users/alessiatosi/Secrets/govuk-bigquery-analytics-service-credentials.json")  

In [ ]:
# Make plots wider 
options(repr.plot.width=15, repr.plot.height=8)

In [ ]:
# create custom plotting theme
theme_custom <- theme(plot.title = element_text(face = "bold", hjust = 0.5, size=18),
                      plot.subtitle = element_text(size=14),
                      axis.text.y = element_text(colour = 'black', size = 12), 
                      axis.title.y = element_text(size = 16, hjust = 0.5, vjust = 0.2),
                      axis.text.x = element_text(colour = 'black', size = 12), 
                      axis.title.x = element_text(size = 16, hjust = 0.5, vjust = 0.2),
                      panel.background = element_blank(),
                      axis.line = element_line(colour = "black"),
                      legend.position = "bottom",
                      legend.direction = "horizontal")

### Get the data

In [ ]:
#billing <- "govuk-xgov" # replace this with your project ID 
project = "govuk-bigquery-analytics"
sql <- "SELECT * FROM `govuk-bigquery-analytics.datascience.related_links_20211011_20211219_intervention`"

tb <- bq_table_download(bq_project_query(project, sql))

### Data pre-processing

In [ ]:
# cast date as a date type variable
tb$date <- as.Date(strptime(tb$date, "%Y%m%d"))

In [ ]:
tb <- tb %>% 
    arrange(date)

In [ ]:
tb

### Plotting the time series of data

Here we will plotting the time series of data for all the metrics. Later in the notebook we will explore seasonality and trends in the time series of the two main evaluation metrics:
- Proportion of visitors who click a related link (RL) at least once
- Proportion of repeated-clicker visitors (those that, having click on a RL, click on others)

In [ ]:
plot_timeseries <- function(data, 
                            ts_var="", 
                            title="", 
                            x_title="", 
                            subtitle = "Intervention time series (visited at least one page with node2vec related links)"){
    #'@param data (data.frame) : dataset  
    #'@param ts_var (character string) : name of the variable containing the time-series data
    #'@param title (character string) : plot title
    #'@param x_title (character string) : x-axis title
    #'@return time-series plot
    
    if(!"date" %in% colnames(data)) stop(paste0("column `date` is missing from dataset"))
    
    sym_ts_var <- dplyr::sym(ts_var)
    
    data %>% 
    ggplot2::ggplot(., aes(date, !!sym_ts_var)) +
    geom_point(size=2) +
    geom_line(size=1) +
    #geom_smooth(method="lm", colour="blue") +
    geom_smooth(method = "loess", formula=y~x, colour="red", se=TRUE) +
    geom_vline(aes(xintercept = as.Date("20211025", "%Y%m%d")), col="blue", linetype=2) +
    geom_vline(aes(xintercept = as.Date("20211110", "%Y%m%d")), col="darkgreen", linetype=2) +
    geom_vline(aes(xintercept = as.Date("20211111", "%Y%m%d")), col="blue", linetype=2) +
    geom_vline(aes(xintercept = as.Date("20211120", "%Y%m%d")), col="purple", linetype=2) +
    geom_vline(aes(xintercept = as.Date("20211211", "%Y%m%d")), col="purple", linetype=2) +
    geom_vline(aes(xintercept = as.Date("20211213", "%Y%m%d")), col="darkgreen", linetype=2) +
    geom_text(aes(x=as.Date("20211025", "%Y%m%d"), y=0, label="25 Oct 2021 unweighted"), size=5, angle=90, vjust=-0.4, hjust=0, color="blue") +
    geom_text(aes(x=as.Date("20211110", "%Y%m%d"), y=0, label="10 Nov 2021 menu bar v2"), size=5, angle=90, vjust=-0.4, hjust=0, color="darkgreen") +
    geom_text(aes(x=as.Date("20211111", "%Y%m%d"), y=0, label="11 Nov 2021 unweighted"), size=5, angle=90, vjust=-0.4, hjust=0, color="blue") +
    geom_text(aes(x=as.Date("20211120", "%Y%m%d"), y=0, label="20 Nov 2021 weighted"), size=5, angle=90, vjust=-0.4, hjust=0, color="purple") +
    geom_text(aes(x=as.Date("20211211", "%Y%m%d"), y=0, label="12 Dec 2021 weighted"), size=5, angle=90, vjust=-0.4, hjust=0, color="purple") +
    geom_text(aes(x=as.Date("20211213", "%Y%m%d"), y=0, label="13 Dec 2021 new homepage"), size=5, angle=90, vjust=-0.4, hjust=0, color="darkgreen") +
    labs(
        title = title,
        subtitle = subtitle) +
    ylab(x_title) +
    theme_custom
    }

#### Main metrics of evaluation

In [ ]:
plot_timeseries(data=tb,
               ts_var="pc_visitors_used_rl",
               title="Proportion of visitors who clicked on at least 1 related link",
               x_title="Proportion of visitors")

Let's check the number to see whether the decline may be explained by an increase in the denominator rather than a decrease in the numerator. This may be the case if, for instance, external circumstance would bring visiotrs to come and look at one page only. 

In [ ]:
plot_timeseries(data=tb,
               ts_var="visitors_that_clicked_rl",
               title="Number of visitors who clicked on at least 1 related link",
               x_title="Number of visitors")

In [ ]:
head(tb)

In [ ]:
plot_timeseries(data=tb,
               ts_var="pc_visitors_returning_to_rl",
               title="Proportion of repeated-clicker visitors (they clicked one RL who then click again on a RL)",
               x_title="Proportion of visitors")

#### Other potential behaviour of interest

In [ ]:
plot_timeseries(data=tb,
               ts_var="pc_visitors_that_clicked_navigation",
               title="Proportion of visitors who clicked on a navigation element",
               x_title="Proportion of visitors")

In [ ]:
plot_timeseries(data=tb,
               ts_var="pc_visitors_2_or_more_rl",
               title="Proportion of visitors who clicked 2 or more related links",
               x_title="Proportion of visitors")

### Trends, seasonality and stationarity

- **Trend**: whether and when there is an overall increasing or decreasing pattern in our observations over time
- **Seasonality**: whether and when there are repeating patterns in the series at fixed and known periods (e.g., weekly)
- **Stationarity**: when a time-series has constant mean, variance and covariance over time
Put another way, a time-series is **stationary** when it has no trend nor seasonality, and has constant variance over time. Typically, this will mean when you plot values over time, it will be roughly horizontal (though some cyclic behaviour is possible) and have constant variance.

- **Remainder/random noise**: leftover of original time-series after trend and seasonality are removed
- **Autocorrelation**: the strength of the relationship between a variable and its observations at prior time-periods
The **autocorrelation function** is a plot of a **stationary** time-series with its lags (meaning its observations at prior time-periods). It can be used to obtain the order of a **moving-average model**, *q*. It will be the first lag at which the **autocorrelation** value passes the upper 95% **confidence interval**, as indicated by the blue dotted line in the corresponding **ACF** plot.

- **Partial autocorrelation**: the strength of the relationship between an observation in a time-series with its observations at prior time-peridos, with the relationships of intervening observations removed. **Partial autocorrelation** is different to **autocorrelation** because the latter is comprised of both *direct* and *indirect* correlations, whereas the former removes these *indirect* correlations. It can be used to obtain the order of an auto-regressive model, *p*.^[Indirect correlations are a linear function of the correlation of the observation, with observations at intervening time periods.]

We explore **trend** to help identify whether the shares in page view traffic by device cateogry has evolved over time, and whether this change in the cookie-policy has further affected this trend in any peculiar way. Whereas for the **ACF** and **PACF** concepts, we explore these to inform our choice of the statistical method to model our time-series data with.


In [ ]:
# convert to time-series object
tb <- tb %>%
    tsibble::as_tsibble(index = date)

In [ ]:
plot_SLT <- function(data, ts_var="", title_ts_var=""){
    #'@param data (data.frame) : dataset  
    #'@param ts_var (character string) : name of the variable containing the time-series data
    #'@param title_ts_var (character string) : Plain English description of time-series variable
    #'@return time-series plot
    
    if(!"date" %in% colnames(data)) stop(paste0("column `date` is missing from dataset"))
    
    sym_ts_var <- dplyr::sym(ts_var)
    
    decomp <- data %>% model(STL(!!sym_ts_var)) %>% components()
    
    p1 <- data %>%
        feasts::gg_tsdisplay(y = !!sym_ts_var, plot_type = "partial") + 
        labs(title = paste(title_ts_var, "- Time, ACF and PACF plots"))
    
    p2 <- decomp %>% autoplot()
    
    list(p1, p2)
    }

#### Proportion of visitors who click RL at least once

In [ ]:
plot_SLT(tb, "pc_visitors_used_rl", "Percentage of visitors who clicked RL at least once")

#### Proportion of repeated-clicker visitors

In [ ]:
plot_SLT(tb, "pc_visitors_returning_to_rl", 
         "Proportion of visitors who clicked on more RLs after having clicked on one")

#### Proportion of visitors who clicked on a navigation element while on a RL page

In [ ]:
plot_SLT(tb, "pc_visitors_that_clicked_navigation", 
         "Proportion of visitors who clicked on a navigation element")

## Conclusions

The time series of our two main metrics of evaluation display both weekly seasonality and some upward not-fully linear trend that we will try to account for when modelling the time-series as part of the interrupted time series analysis.

### Comparing whole intervention time-series (pre-/post segments) and control

In [ ]:
# get the intervention data
sql_control <- "SELECT * FROM `govuk-bigquery-analytics.datascience.control_manual_links_20211011_20211219_data`"

tb_control <- bq_table_download(bq_project_query(project, sql_control))
tb_control$date <- as.Date(strptime(tb_control$date, "%Y%m%d"))
tb_control <- tb_control %>% 
    arrange(date)

In [ ]:
# join the two time series by date
ts <- tb %>% inner_join(tb_control, by="date")

In [ ]:
names(ts)

In [ ]:
ts <- ts[, c('date', 
             'visitors.x',
             'visitors.y', 
             'visitors_that_clicked_rl.x',
             'visitors_that_clicked_rl.y',
             'visitors_2_or_more_rl.x',
             'visitors_2_or_more_rl.y',
             'pc_visitors_used_rl.x',
             'pc_visitors_used_rl.y',
             'pc_visitors_returning_to_rl.x',
             'pc_visitors_returning_to_rl.y')]

In [ ]:
ts_long <- gather(ts, metric, value, c('visitors.x',
                                       'visitors.y', 
                                       'visitors_that_clicked_rl.x',
                                       'visitors_that_clicked_rl.y',
                                       'visitors_2_or_more_rl.x',
                                       'visitors_2_or_more_rl.y',
                                       'pc_visitors_used_rl.x',
                                       'pc_visitors_used_rl.y',
                                       'pc_visitors_returning_to_rl.x',
                                       'pc_visitors_returning_to_rl.y'), 
                   factor_key=FALSE)



In [ ]:
tail(ts_long)

In [ ]:
ts_long <- ts_long %>% 
    separate(metric, c("metric", "time_series"), sep = "\\.") %>% 
    mutate(time_series = ifelse(time_series == "x", "intervention", "control"))   

In [ ]:
head(ts_long)


In [ ]:
plot_multiple_timeseries <- function(data, 
                            ts_var="", 
                            group_var="",
                            title="", 
                            x_title="", 
                            subtitle = "Control-group time series (only visited pages with manually-curated links )"){
    #'@param data (data.frame) : dataset  
    #'@param ts_var (character string) : name of the variable containing the time-series data
    #'@param title (character string) : plot title
    #'@param x_title (character string) : x-axis title
    #'@return time-series plot
    
    if(!"date" %in% colnames(data)) stop(paste0("column `date` is missing from dataset"))
    
    sym_ts_var <- dplyr::sym(ts_var)
    sym_group_var <- dplyr::sym(group_var)
    
    data %>% 
    ggplot2::ggplot(., aes(date, !!sym_ts_var, group=!!sym_group_var, colour=!!sym_group_var)) +
    geom_point(size=2) +
    geom_line(size=1) +
    #geom_smooth(method="lm", colour="blue") +
    geom_smooth(method = "loess", formula=y~x, colour="red", se=TRUE) +
    geom_vline(aes(xintercept = as.Date("20211025", "%Y%m%d")), col="blue", linetype=2) +
    geom_vline(aes(xintercept = as.Date("20211110", "%Y%m%d")), col="darkgreen", linetype=2) +
    geom_vline(aes(xintercept = as.Date("20211111", "%Y%m%d")), col="blue", linetype=2) +
    geom_vline(aes(xintercept = as.Date("20211120", "%Y%m%d")), col="purple", linetype=2) +
    geom_vline(aes(xintercept = as.Date("20211211", "%Y%m%d")), col="purple", linetype=2) +
    geom_vline(aes(xintercept = as.Date("20211213", "%Y%m%d")), col="darkgreen", linetype=2) +
    geom_text(aes(x=as.Date("20211025", "%Y%m%d"), y=0, label="25 Oct 2021 unweighted"), size=5, angle=90, vjust=-0.4, hjust=0, color="blue") +
    geom_text(aes(x=as.Date("20211110", "%Y%m%d"), y=0, label="10 Nov 2021 menu bar v2"), size=5, angle=90, vjust=-0.4, hjust=0, color="darkgreen") +
    geom_text(aes(x=as.Date("20211111", "%Y%m%d"), y=0, label="11 Nov 2021 unweighted"), size=5, angle=90, vjust=-0.4, hjust=0, color="blue") +
    geom_text(aes(x=as.Date("20211120", "%Y%m%d"), y=0, label="20 Nov 2021 weighted"), size=5, angle=90, vjust=-0.4, hjust=0, color="purple") +
    geom_text(aes(x=as.Date("20211211", "%Y%m%d"), y=0, label="12 Dec 2021 weighted"), size=5, angle=90, vjust=-0.4, hjust=0, color="purple") +
    geom_text(aes(x=as.Date("20211213", "%Y%m%d"), y=0, label="13 Dec 2021 new homepage"), size=5, angle=90, vjust=-0.4, hjust=0, color="darkgreen") +
    labs(
        title = title,
        subtitle = subtitle) +
    ylab(x_title) +
    theme_custom
    }

In [ ]:
ts_long %>%
    filter(metric=="pc_visitors_used_rl") %>%
    plot_multiple_timeseries(data=., 
                         ts_var="value", 
                         group_var="time_series",
                         title="Proportion of day visitors who clicked on at least 1 related-link", 
                         x_title="Proportion of day visitors", 
                         subtitle = "Intervention and control time series")

In [ ]:
ts_long %>%
    filter(metric=="pc_visitors_returning_to_rl") %>%
    plot_multiple_timeseries(data=., 
                         ts_var="value", 
                         group_var="time_series",
                         title="Proportion of daily repeated-clicker visitors", 
                         x_title="Proportion of daily visitors", 
                         subtitle = "Intervention and control time series")

In [ ]:
unique(ts_long$metric)

### Possible concurring cause

Here we will check whether an increase in single page visits, perhaps linked to an increased needs to find coronavirus information like reporting lateral flow test or checking travel advice, may explain the decrease in proportions of users who click on related links.

Single-page visiting visitors, in fact, would "inflate" the denominator in our proportions.

In [ ]:
sql_singlepage_sessions <- "SELECT * FROM `govuk-bigquery-analytics.datascience.singlepage_dayvisitors_20211011_20211219`"

tb_ss <- bq_table_download(bq_project_query(project, sql_singlepage_sessions))

In [ ]:
# cast date as a date type variable
tb_ss$date <- as.Date(strptime(tb_ss$date, "%Y%m%d"))

# filter for the single-page session data
tb_ss <- tb_ss %>%
    filter(single_pagehit_session == TRUE)


In [ ]:
plot_timeseries(data=tb_ss, 
                ts_var="proportion", 
                x_title="Proportion of single-page day-visitors", 
                title="Proportion of visitors who only visited one page on given day",
               subtitle="")

In [ ]:
plot_timeseries(data=tb_ss, 
                ts_var="frequency",
                x_title="Number of single-page day-visitors", 
                title="Number of visitors who only visited one page on given day",
               subtitle="")

## Analysis

### Proportion of visitors who clicked on at least one related link

In [ ]:
clicks_ts <- ts_long %>% filter(metric=='pc_visitors_used_rl') 

Create new variables:
- **Time**: time variable capturing time passed from start of the intervention
- **Intervention**: dummy variable signalling before (0) and after (1) weighted related-links were introduced
- **TimeSince**: variable capturing time passed since intervention (introduction of weighted related links) occured

In [ ]:
clicks_ts <- clicks_ts %>%
    group_by(time_series) %>%
    mutate(Time = row_number()) %>%
    mutate(Intervention = ifelse(date <= as.Date("2021-11-20"), 0, 1))

In [ ]:
# how many data points after intervention took place?
table(clicks_ts[clicks_ts$time_series=="intervention",]$Intervention)

In [ ]:
clicks_ts$TimeSince <- 0
clicks_ts[clicks_ts$time_series=="intervention",]$TimeSince <- c(rep(0, 41), rep(1:29))
clicks_ts[clicks_ts$time_series=="control",]$TimeSince <- c(rep(0, 41), rep(1:29))

In [ ]:
# intervention only data
clicks_its <- clicks_ts[clicks_ts$time_series=="intervention",]

In [ ]:
head(clicks_its)

In [ ]:
# plot
plot( clicks_its$Time, 
      clicks_its$value,
      bty="n", pch=19, col="gray",
      ylim = c(0, 0.04), xlim=c(0,72),
      xlab = "Time (days)", 
      ylab = "pc_visitors_used_rl" )

# Line marking the interruption
abline( v=42, col="firebrick", lty=2 )
text( 42, 0, "Start of Weighted related links", col="firebrick", cex=1.3, pos=4 )

# Add the regression line
ts <- lm( value ~ Time + Intervention + TimeSince, data=clicks_its )
lines( clicks_its$Time, ts$fitted.values, col="steelblue", lwd=2 )

In [ ]:
reg_clicks <- lm( value ~ Time + Intervention + TimeSince, data=clicks_its )

pred1 <- predict(reg_clicks, clicks_its) 
# To estimate all predicted values of Y, we just use our dataset

datanew <- as.data.frame(cbind(Time = rep(1 : length(clicks_its$Time)), Intervention = rep(0), TimeSince = rep(0))) 
# Create a new dataset where Treatment and Time Since Treatment are equal to 0 as the intervention did not occur.
pred2 <- predict(reg_clicks, datanew) 
# Predict the counterfactuals

plot(clicks_its$Time, 
    clicks_its$value,
      bty="n",
      col = gray(0.3,0.5), pch=19,
      ylim = c(0, 0.04), 
      xlim = c(0,72),
      main = "Proportion of visitors who clicked on at least 1 related link",
      xlab = "Time elapsed since beginning of study (days)", 
      ylab = "Proportion of visitors")

lines( 1:41, pred1[1:41], col="dodgerblue4", lwd = 3 )
lines( 42:70, pred1[42:70], col="dodgerblue4", lwd = 3 )
lines( 42:70, pred2[42:70], col="darkorange2", lwd = 3, lty = 5 ) 

text(45, 0.015, labels = "Predicted values", pos = 4, cex = 1.3, col = "dodgerblue3")
text(50, 0.025, labels = "Counterfactual", pos = 4, cex = 1.3, col = "darkorange2")

# Line marking the interruption
abline( v=42, col="firebrick",lwd = 2, lty=2 )
text( 42, 0, "Start of weighted related links", col="firebrick", cex=1.3, pos=4 )


In [ ]:
summary(reg)

In [ ]:
acf(resid( reg ))

#### Poisson approach

In [ ]:
unique(ts_long$metric)

In [ ]:
clicks_ts <- ts_long %>% filter(metric %in% c('visitors', 'visitors_that_clicked_rl')) 


clicks_ts <- clicks_ts %>%
    group_by(time_series, metric) %>%
    mutate(Time = row_number()) %>%
    mutate(Intervention = ifelse(date <= as.Date("2021-11-20"), 0, 1))

In [ ]:
# check
table(clicks_ts[clicks_ts$metric == "visitors",]$time_series)
table(clicks_ts[clicks_ts$metric == "visitors_that_clicked_rl",]$time_series)

In [ ]:
hist(clicks_ts[clicks_ts$metric == "visitors_that_clicked_rl",]$value, breaks=20)

In [ ]:
hist(clicks_ts[clicks_ts$metric == "visitors",]$value, breaks=20)

In [ ]:
clicks_ts_spread <- clicks_ts %>%
    spread(metric, value)

In [ ]:
clicks_ts_spread$TimeSince <- 0
clicks_ts_spread[clicks_ts_spread$time_series=="intervention",]$TimeSince <- c(rep(0, 41), rep(1:29))
clicks_ts_spread[clicks_ts_spread$time_series=="control",]$TimeSince <- c(rep(0, 41), rep(1:29))

In [ ]:
clicks_ts_spread_int <- clicks_ts_spread[clicks_ts_spread$time_series == "intervention",]

In [ ]:
poisson_reg <- glm(visitors_that_clicked_rl ~ Time + Intervention + TimeSince + offset(log(visitors)), 
                    data=clicks_ts_spread_int, 
                    family = poisson(link = "log"))

In [ ]:
summary(poisson_reg)

##### Overdispersion...

The Residual Deviance is greater than the degrees of freedom, thus over-dispersion exists. This means that the estimates are correct, but the standard errors are wrong and unaccounted for by the model.

Let's use a quasi-poisson link family for the error to try to correct for this.

In [ ]:
quasipoisson_reg <- glm(visitors_that_clicked_rl ~ Time + Intervention + TimeSince + offset(log(visitors)), 
                    data=clicks_ts_spread_int, 
                    family = quasipoisson(link = "log"))

In [ ]:
summary(quasipoisson_reg)

In [ ]:
# let's check residuals for evidence of autocorrelation/ seasonality
acf(residuals(quasipoisson_reg, type='deviance'))

Ok, the familiar day-of-the-week seasonality kicks in... we should account for it in the model. 

In [ ]:
summary(quasipoisson_reg)$dispersion
round(ci.lin(quasipoisson_reg, Exp=T),3)

In [ ]:
clicks_ts_spread_int$ctr <- with(clicks_ts_spread_int, visitors_that_clicked_rl/visitors*10^5)

In [ ]:
head(clicks_ts_spread_int)

In [ ]:
### NEED TO FIX THIS PLOT, SOMETHING IS NOT RIGHT

pred1 <- predict(quasipoisson_reg_season, clicks_ts_spread_int, type = "response")/mean(
    clicks_ts_spread_int$visitors)*10^5
# To estimate all predicted values of Y, we just use our dataset

# Predict the counterfactuals
# Create a new dataset where Treatment and Time Since Treatment are equal to 0 as the intervention did not occur.
datanew <- as.data.frame(cbind(
    visitors = mean(clicks_ts_spread_int$visitors),
    Time = 1:length(clicks_ts_spread_int$Time), 
    Intervention = 0, 
    TimeSince = 0)) 
pred2 <- predict(quasipoisson_reg_season, datanew, type = "response")/mean(
    clicks_ts_spread_int$visitors)*10^5 

plot(clicks_ts_spread_int$ctr, 
      bty="n",
      col = gray(0.3,0.5), pch=19,
      #ylim = c(0, 46000), 
      xlim = c(0,72),
      main = "Proportion of visitors who clicked on at least 1 related link",
      xlab = "Time elapsed since beginning of study (days)", 
      ylab = "Proportion of visitors")


#lines( 1:41, pred1[1:41], col="dodgerblue4", lwd = 3 )
#lines( 42:70, pred1[42:70], col="dodgerblue4", lwd = 3 )
lines(1:70,pred1,col="dodgerblue4")
#lines( 42:70, pred2[42:70], col="darkorange2", lwd = 3, lty = 5 ) 

lines(datanew$Time,pred2,col="darkorange2",lty=2)


text(45, 0.015, labels = "Predicted values", pos = 4, cex = 1.3, col = "dodgerblue3")
text(50, 0.025, labels = "Counterfactual", pos = 4, cex = 1.3, col = "darkorange2")

# Line marking the interruption
abline( v=42, col="firebrick",lwd = 2, lty=2 )
text( 42, 0, "Start of weighted related links", col="firebrick", cex=1.3, pos=4 )


##### Using Fourier terms to adjust for seasonlity, using sine and cosine transform functions

These are periodic terms composed of sine and cosine waves that can model regular fluctuations, such as those from seasonal variation

In [ ]:
#c) adjusting for seasonality
# There are various ways of adjusting for seasonality - here we use harmonic
#   terms specifying the number of sin and cosine pairs to include (in this
#   case 2) and the length of the period (7 days)
quasipoisson_reg_season <- glm(visitors_that_clicked_rl ~ Time + Intervention + TimeSince + 
                               offset(log(visitors)) +
                               tsModel::harmonic(clicks_ts_spread_int$date,2,7), 
                    data=clicks_ts_spread_int, 
                    family = quasipoisson(link = "log"))



In [ ]:
summary(quasipoisson_reg_season)

In [ ]:
acf(residuals(quasipoisson_reg_season, type='deviance'))

In [ ]:
pacf(residuals(quasipoisson_reg_season, type='deviance'))

There is no longer seasonality, still some autocorrelation though.

Thus, we keep the quasi-Poisson model fitted ignoring the autocorrelation but adjust the
standard errors using the Newey-West method (ref: https://researchonline.lshtm.ac.uk/id/eprint/4651093/1/Analysing-interrupted-time-series-with-a-control.pdf)

In [ ]:
coeftest(quasipoisson_reg_season, vcov = NeweyWest(quasipoisson_reg_season, lag=4))

In [ ]:
NeweyWest(quasipoisson_reg_season, lag = 2, prewhite = F) %>%
    diag() %>% sqrt()

In [ ]:
glm(visitors_that_clicked_rl ~ Time + Intervention + TimeSince + 
                               offset(log(visitors)) +
                               tsModel::harmonic(clicks_ts_spread_int$date,2,7), 
                    data=clicks_ts_spread_int, 
                    family = quasipoisson(link = "log"))

In [ ]:
reg_clicks <- lm( ctr ~ Time + Intervention + TimeSince +
                tsModel::harmonic(clicks_ts_spread_int$date,2,7), 
                    data=clicks_ts_spread_int)

pred1 <- predict(reg_clicks, clicks_ts_spread_int) 
# To estimate all predicted values of Y, we just use our dataset

datanew <- as.data.frame(cbind(Time = rep(1 : length(clicks_its$Time)), Intervention = rep(0), TimeSince = rep(0))) 
# Create a new dataset where Treatment and Time Since Treatment are equal to 0 as the intervention did not occur.
pred2 <- predict(reg_clicks, datanew)
# Predict the counterfactuals

plot(clicks_ts_spread_int$ctr, 
      bty="n",
      col = gray(0.3,0.5), pch=19,
      #ylim = c(0, 0.04), 
      xlim = c(0,72),
      main = "Rate of visitors who clicked on at least 1 related link (per 10000 visitors)",
      xlab = "Time elapsed since beginning of study (days)", 
      ylab = "Rate of visitors (per 10,000)")

lines( 1:41, pred1[1:41], col="dodgerblue4", lwd = 3 )
lines( 42:70, pred1[42:70], col="dodgerblue4", lwd = 3 )
lines( 42:70, pred2[42:70], col="darkorange2", lwd = 3, lty = 5 ) 

text(45, 1500, labels = "Predicted values", pos = 4, cex = 1.3, col = "dodgerblue3")
text(50, 2700, labels = "Counterfactual", pos = 4, cex = 1.3, col = "darkorange2")

# Line marking the interruption
abline( v=42, col="firebrick",lwd = 2, lty=2 )
text( 42, 1200, "Start of weighted related links", col="firebrick", cex=1.3, pos=4 )


In [ ]:
summary(reg_clicks)

In [ ]:
plot(acf(resid(reg_clicks)))

In [ ]:
coeftest(reg_clicks, vcov = NeweyWest(reg_clicks, lag=2))

#### Controlled Interrupted Time Series

In [ ]:
clicks_ts_spread <- clicks_ts_spread %>%
    mutate(ctr = visitors_that_clicked_rl/visitors*10^5)

In [ ]:
ctr_ts_spread <- clicks_ts_spread %>%
    select(date, Time, Intervention, TimeSince, ctr) %>%
    spread(time_series, ctr)

In [ ]:
head(ctr_ts_spread)

In [ ]:
ctr_ts_spread$ctr_diff <- with(ctr_ts_spread, intervention-control)

In [ ]:
head(ctr_ts_spread)

In [ ]:
plot(ctr_ts_spread$ctr_diff, 
      bty="n",
      col = gray(0.3,0.5), pch=19,
      #ylim = c(0, 0.04), 
      xlim = c(0,72),
      main = "Difference in Rate of visitors who clicked on at least 1 related link (per 10000 visitors)",
      xlab = "Time elapsed since beginning of study (days)", 
      ylab = "Difference in Rate of visitors (per 10,000)")
# Line marking the interruption
abline( v=42, col="firebrick",lwd = 2, lty=2 )
text( 42, 350, "Start of weighted related links", col="firebrick", cex=1.3, pos=4 )


In [ ]:
reg_ctr_diff <- lm( ctr_diff ~ Time + Intervention + TimeSince +
                tsModel::harmonic(ctr_ts_spread$date,2,7), 
                    data=ctr_ts_spread)


In [ ]:
summary(reg_ctr_diff)

In [ ]:
plot(acf(resid(reg_ctr_diff)))

In [ ]:
# robust standrad error
coeftest(reg_ctr_diff, vcov = NeweyWest(reg_ctr_diff, lag=2))

In [ ]:
diff_pred1 <- predict(reg_ctr_diff, ctr_ts_spread) 
# To estimate all predicted values of Y, we just use our dataset

diff_datanew <- as.data.frame(cbind(Time = rep(1 : length(ctr_ts_spread$Time)), 
                                    Intervention = 0, 
                                    TimeSince = 0)) 
# Create a new dataset where Treatment and Time Since Treatment are equal to 0 as the intervention did not occur.
diff_pred2 <- predict(reg_ctr_diff, diff_datanew)
# Predict the counterfactuals

plot(ctr_ts_spread$ctr_diff, 
      bty="n",
      col = gray(0.3,0.5), pch=19,
      #ylim = c(0, 0.04), 
      xlim = c(0,72),
      main = "Difference in Rate of visitors who clicked on at least 1 related link (per 100,000 visitors)",
      xlab = "Time elapsed since beginning of study (days)", 
      ylab = "Difference in Rate of visitors (per 100,000)")

lines( 1:41, diff_pred1[1:41], col="dodgerblue4", lwd = 3 )
lines( 42:70, diff_pred2[42:70], col="dodgerblue4", lwd = 3 )
lines( 42:70, diff_pred2[42:70], col="darkorange2", lwd = 3, lty = 5 ) 

text(45, 1500, labels = "Predicted values", pos = 4, cex = 1.3, col = "dodgerblue3")
text(50, 2700, labels = "Counterfactual", pos = 4, cex = 1.3, col = "darkorange2")

# Line marking the interruption
abline( v=42, col="firebrick",lwd = 2, lty=2 )
text( 42, 350, "Start of weighted related links", col="firebrick", cex=1.3, pos=4 )


So after accounting for confounding variables via the controlled time series, we can see there is actually no effect whatsoever due to our intervention

### Proportion of repeated-clicker visitors

Let's do the same for our second evaluation metric

In [ ]:
unique(ts_long$metric)

In [ ]:
repeat_ts <- ts_long %>% filter(metric %in% c('visitors_that_clicked_rl', 'visitors_2_or_more_rl')) 


repeat_ts <- repeat_ts %>%
    group_by(time_series, metric) %>%
    mutate(Time = row_number()) %>%
    mutate(Intervention = ifelse(date <= as.Date("2021-11-20"), 0, 1))

In [ ]:
# check
table(repeat_ts[repeat_ts$metric == "visitors_2_or_more_rl",]$time_series)
table(repeat_ts[repeat_ts$metric == "visitors_that_clicked_rl",]$time_series)

In [ ]:
head(repeat_ts)

In [ ]:
repeat_ts_spread <- repeat_ts %>%
    spread(metric, value)

In [ ]:
repeat_ts_spread$TimeSince <- 0
repeat_ts_spread[repeat_ts_spread$time_series=="intervention",]$TimeSince <- c(rep(0, 41), rep(1:29))
repeat_ts_spread[repeat_ts_spread$time_series=="control",]$TimeSince <- c(rep(0, 41), rep(1:29))

In [ ]:
repeat_ts_spread$repeated_ctr <- with(repeat_ts_spread, visitors_2_or_more_rl/visitors_that_clicked_rl*10^3)

In [ ]:
head(repeat_ts_spread)

In [ ]:
repeat_ts_int <- repeat_ts_spread %>% filter(time_series=="intervention")

In [ ]:
repeat_ts_int[38:44,]

### ITS analysis

We use the same approach and run an LSO on the rate values (per 10,000) (we can confirm with a quasipoisson as above later if needed), modelling seasonality with Fourier terms and autocorrelated errors with robust NeweyWest standard errors.

In [ ]:
plot(repeat_ts_int$repeated_ctr, 
      bty="n",
      col = gray(0.3,0.5), pch=19,
      xlim = c(0,72),
      main = "Rate of repeated-click visitors (per 1,000 visitors)",
      xlab = "Time elapsed since beginning of study (days)", 
      ylab = "Rate of repeated-click visitors (per 1,000)")

# Line marking the interruption
abline( v=42, col="firebrick",lwd = 2, lty=2 )
text( 42, 160, "Start of weighted related links", col="firebrick", cex=1.3, pos=4 )


In [ ]:
reg_repeated <- lm( repeated_ctr ~ Time + Intervention + TimeSince +
                tsModel::harmonic(repeat_ts_int$date,2,7), 
                    data=repeat_ts_int)


In [ ]:
plot(acf(resid(reg_repeated)))

In [ ]:
# robust standrad error
coeftest(reg_repeated, vcov = NeweyWest(reg_repeated, lag=2))

No effect of our intervention (switch to related links), just a slight pre-intervention increase seemingly unaffacted by our intervention) 

In [ ]:
rep_pred1 <- predict(reg_repeated, repeat_ts_int) 
# To estimate all predicted values of Y, we just use our dataset

rep_datanew <- as.data.frame(cbind(
    Time = rep(1 : length(repeat_ts_int$Time)), 
    Intervention = 0, 
    TimeSince = 0)) 
# Create a new dataset where Treatment and Time Since Treatment are equal to 0 as the intervention did not occur.
rep_pred2 <- predict(reg_repeated, rep_datanew)
# Predict the counterfactuals

plot(repeat_ts_int$repeated_ctr, 
      bty="n",
      col = gray(0.3,0.5), pch=19,
      ylim = c(170, 250), 
      xlim = c(0,72),
      main = "Rate of repeated-clicker visitors (per 1,000 visitors)",
      xlab = "Time elapsed since beginning of study (days)", 
      ylab = "Rate of visitors (per 1,000)")

lines( 1:41, rep_pred1[1:41], col="dodgerblue4", lwd = 3 )
lines( 42:70, rep_pred1[42:70], col="dodgerblue4", lwd = 3 )
lines( 42:70, rep_pred2[42:70], col="darkorange2", lwd = 3, lty = 5 ) 

text(45, 190, labels = "Predicted values", pos = 4, cex = 1.3, col = "dodgerblue3")
text(50, 230, labels = "Counterfactual", pos = 4, cex = 1.3, col = "darkorange2")

# Line marking the interruption
abline( v=42, col="firebrick",lwd = 2, lty=2 )
text( 42, 170, "Start of weighted related links", col="firebrick", cex=1.3, pos=4 )


### Controlled time-series

In [ ]:
repeat_ts_spread <- repeat_ts_spread %>%
    mutate(ctr = visitors_that_clicked_rl/visitors*10^5)

In [ ]:
diff_repeat_ts <- repeat_ts_spread %>%
    select(date, time_series, Time, Intervention, TimeSince, repeated_ctr) %>%
    spread(time_series, repeated_ctr)

In [ ]:
head(diff_repeat_ts)

In [ ]:
diff_repeat_ts$repeat_diff <- with(diff_repeat_ts, intervention-control)

In [ ]:
head(diff_repeat_ts)

In [ ]:
plot(diff_repeat_ts$repeat_diff, 
      bty="n",
      col = gray(0.3,0.5), pch=19,
      #ylim = c(170, 250), 
      xlim = c(0,72),
      main = "Difference in rate of repeated-clicker visitors (per 1,000 visitors)",
      xlab = "Time elapsed since beginning of study (days)", 
      ylab = "Difference in rate of visitors (per 1,000)")

# Line marking the interruption
abline( v=42, col="firebrick",lwd = 2, lty=2 )
text( 42, 29, "Start of weighted related links", col="firebrick", cex=1.3, pos=4 )


In [ ]:
reg_repeat_diff <- lm( repeat_diff ~ Time + Intervention + TimeSince +
                tsModel::harmonic(diff_repeat_ts$date,2,7), 
                    data=diff_repeat_ts)


In [ ]:
summary(reg_repeat_diff)

In [ ]:
plot(acf(resid(reg_repeat_diff)))

In [ ]:
# robust standrad error
coeftest(reg_repeat_diff, vcov = NeweyWest(reg_repeat_diff, lag=3))

In [ ]:
rep_diff_pred1 <- predict(reg_repeat_diff, diff_repeat_ts) 
# To estimate all predicted values of Y, we just use our dataset

rep_diff_datanew <- as.data.frame(cbind(
    Time = rep(1 : length(diff_repeat_ts$Time)), 
    Intervention = 0, 
    TimeSince = 0)) 
# Create a new dataset where Treatment and Time Since Treatment are equal to 0 as the intervention did not occur.
rep_diff_pred2 <- predict(reg_repeat_diff, rep_diff_datanew)
# Predict the counterfactuals

plot(diff_repeat_ts$repeat_diff, 
      bty="n",
      col = gray(0.3,0.5), pch=19,
      ylim = c(0, 70), 
      xlim = c(0,72),
      main = "Difference in rate of repeated-clicker visitors (per 1,000 visitors)",
      xlab = "Time elapsed since beginning of study (days)", 
      ylab = "Difference in rate of visitors (per 1,000)")

lines( 1:41, rep_diff_pred1[1:41], col="dodgerblue4", lwd = 3 )
lines( 41:70, rep_diff_pred1[41:70], col="dodgerblue4", lwd = 3 )
lines( 42:70, rep_diff_pred2[42:70], col="darkorange2", lwd = 3, lty = 5 ) 

text(45, 38, labels = "Predicted values", pos = 4, cex = 1.3, col = "dodgerblue3")
text(50, 68, labels = "Counterfactual", pos = 4, cex = 1.3, col = "darkorange2")

# Line marking the interruption
abline( v=42, col="firebrick",lwd = 2, lty=2 )
text( 42, 0, "Start of weighted related links", col="firebrick", cex=1.3, pos=4 )


Here again, no expected effect of our intervention on the time series of the evaluation metric.